In [1]:
import os
import unittest
import pytest
import pandas as pd
import numpy as np

import pyfvs
from pyfvs import fvs

In [2]:
variants = [('pnc',), ('wcc',), ('soc',), ('cac',), ('ecc',), ('oc',), ('op',)]

# root = os.path.split(__file__)[0]
root = r'C:\workspace\forest-modeling\PyFVS\pyfvs\test'
# bare_ground_params = [
#         ['pn', 'rmrs/pn_bareground.key', 'rmrs/pn_bareground.sum.save'],
#         ['wc', 'rmrs/wc_bareground.key', 'rmrs/wc_bareground.sum.save'],
#         ['so', 'rmrs/so_bareground.key', 'rmrs/so_bareground.sum.save'],
#         ['ca', 'rmrs/ca_bareground.key', 'rmrs/ca_bareground.sum.save'],
#         ['ec', 'rmrs/ec_bareground.key', 'rmrs/ec_bareground.sum.save'],
#         ['oc', 'rmrs/oc_bareground.key', 'rmrs/oc_bareground.sum.save'],
#         ['op', 'rmrs/op_bareground.key', 'rmrs/op_bareground.sum.save'],
#         ]

variant = 'oc'
kwd_path = f'{root}/rmrs/{variant}_bareground.key'
sum_path = f'{root}/rmrs/{variant}_bareground.sum.save'

In [3]:
f = fvs.FVS(variant)

f.init_projection(os.path.join(root, kwd_path))
f.tree_data.live_tpa[:, :] = 0.0

for c in range(f.globals.ncyc):
    r = f.grow_projection()

r = f.end_projection()
assert r == 0

In [4]:
widths = [4, 4, 6, 4, 5, 4, 4, 5, 6, 6, 6, 6, 6, 6, 6, 4, 5, 4, 4, 5, 8, 5, 6, 8, 4, 2, 1]
fldnames = (
        'year,age,tpa,baa,sdi,ccf,top_ht,qmd,total_cuft'
        ',merch_cuft,merch_bdft,rem_tpa,rem_total_cuft'
        ',rem_merch_cuft,rem_merch_bdft,res_baa,res_sdi'
        ',res_ccf,res_top_ht,resid_qmd,grow_years'
        ',annual_acc,annual_mort,mai_merch_cuft,for_type'
        ',size_class,stocking_class'
        ).split(',')

# Read the sum file generated by the "official" FVS executable
sum_check = pd.read_fwf(os.path.join(root, sum_path), skiprows=0, widths=widths)
sum_check.columns = fldnames

ncyc = f.globals.ncyc

In [5]:
pd.Series(f.tree_data.cut_tpa[:, 4]).to_clipboard()

In [11]:
# Live TPA
live_tpa = np.sum(f.tree_data.live_tpa[:, :ncyc + 1], axis=0)
check_tpa = sum_check.loc[:, 'tpa'].values
print(np.round(live_tpa, 0).astype(int))
print(check_tpa)
assert np.all(np.isclose(check_tpa, live_tpa, atol=1))


[  0 450 440 430 419 178 176 173 170 167 164]
[  0 450 440 430 419 178 176 173 170 167 164]


In [12]:
# Cut BdFt +/- 1
cut_tpa = f.tree_data.cut_tpa[:, :ncyc + 1]
bdft = f.tree_data.bdft_net[:, :ncyc + 1]
cut_bdft = np.round(np.sum(cut_tpa * bdft, axis=0), 0).astype(int)
check_cut = sum_check.loc[:, 'rem_merch_bdft'].values
print(np.round(np.sum(cut_tpa, axis=0), 0).astype(int))
print(cut_bdft)
print(check_cut)
assert np.all(np.isclose(check_cut, cut_bdft, atol=1))

[  0   0   0   0 239   0   0   0   0   0   0]
[   0    0    0    0 6218    0    0    0    0    0    0]
[   0    0    0    0 6218    0    0    0    0    0    0]


In [13]:
f.tree_data.cut_tpa.sum(axis=0).round(0).astype(int)

array([  0,   0,   0,   0, 239,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])